In [1]:
#Install TensorFlow
!pip install -q tensorflow

import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
# More imports
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, \
  Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/Users/chriskamerin/Desktop/Shopping_Recommender_System/headphones5.csv')

In [4]:
df1 = pd.read_csv('/Users/chriskamerin/Desktop/Shopping_Recommender_System/headphones2hp.csv')

In [5]:
df2 = pd.read_csv('/Users/chriskamerin/Desktop/Shopping_Recommender_System/df2.csv')

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df1 = df1.sample(frac=1).reset_index(drop=True)
df2 = df2.sample(frac=1).reset_index(drop=True)

In [7]:
# We can't trust the userId and movieId to be numbered 0...N-1
# Let's just set our own ids

# current_user_id = 0
# custom_user_map = {} # old user id > new user id
# def map_user_id(row):
#   global current_user_id, custom_user_map
#   old_user_id = row['userId']
#   if old_user_id not in custom_user_map:
#     custom_user_map[old_user_id] = current_user_id
#     current_user_id += 1
#   return custom_user_map[old_user_id]

# df['new_user_id'] = df.apply(map_user_id, axis=1)

df['user_id']  = pd.Categorical(df.user_id).codes
df1['user_id']  = pd.Categorical(df1.user_id).codes
df2['user_id']  = pd.Categorical(df2.user_id).codes
# 

In [8]:
# Now do the same thing for movie ids
# current_movie_id = 0
# custom_movie_map = {} # old movie id > new movie id
# def map_movie_id(row):
#   global current_movie_id, custom_movie_map
#   old_movie_id = row['movieId']
#   if old_movie_id not in custom_movie_map:
#     custom_movie_map[old_movie_id] = current_movie_id
#     current_movie_id += 1
#   return custom_movie_map[old_movie_id]

# df['new_movie_id'] = df.apply(map_movie_id, axis=1)

df['headphone_name']  = pd.Categorical(df.headphone_name).codes
df1['headphone_name']  = pd.Categorical(df1.headphone_name).codes
df2['headphone_name']  = pd.Categorical(df2.headphone_name).codes
# 

In [9]:
cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance','noise_canceling_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover', 'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
       'price_sensitivity']
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']




In [10]:
print('feat col min:' + str(min(df[hp_feat_col].min())),'feat col max:'+ str(max(df[hp_feat_col].max())),
'rem col min:'+ str(min(df[remaining_col].min())),'rem col max:'+str(max(df[remaining_col].max())))

feat col min:0.0 feat col max:10.0 rem col min:0 rem col max:300


In [11]:
df=df.fillna(0)
df[imp_col] = df[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df[price_col] = df[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df[hp_feat_col] = df[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))   
df[remaining_col] = df[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))       
#df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [12]:
cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
        'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
       'price_sensitivity']
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']

   

In [13]:
print('feat col min:' + str(min(df1[hp_feat_col].min())),'feat col max:'+ str(max(df1[hp_feat_col].max())),
'rem col min:'+ str(min(df1[remaining_col].min())),'rem col max:'+str(max(df1[remaining_col].max())))

feat col min:0.0 feat col max:9.2 rem col min:0 rem col max:300


In [14]:
df1=df1.fillna(0)
df1[imp_col] = df1[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df1[price_col] = df1[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df1[hp_feat_col] = df1[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df1[remaining_col] = df1[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min())) 

In [15]:
cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
        'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
       'price_sensitivity']
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']

 

In [16]:
print('feat col min:' + str(min(df2[hp_feat_col].min())),'feat col max:'+ str(max(df2[hp_feat_col].max())),
'rem col min:'+ str(min(df2[remaining_col].min())),'rem col max:'+str(max(df2[remaining_col].max())))

feat col min:0.0 feat col max:0.9999972902885731 rem col min:10.0 rem col max:300.0


In [17]:
df2=df2.fillna(0)
df2[imp_col] = df2[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df2[price_col] = df2[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df2[hp_feat_col] = df2[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df2[remaining_col] = df2[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))  

In [18]:
X=df[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][:70]
Y=df[[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][:70]

In [19]:
x=df1[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']]
y=df1[[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]

In [20]:
randx=df2[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][:8000]
randy=df2[[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][:8000]

In [21]:
lreg = LinearRegression()
lreg1 = LinearRegression()
lreg2 = LinearRegression()

In [22]:
lreg.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
lreg1.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
lreg2.fit(randx,randy)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:


xpred = df[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][70:]
ypred = df[[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][70:]

randxpred = df2[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][80:]
randypred = df2[[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][80:]

In [26]:
ypred = ypred.reset_index()
ypred = ypred.drop("index",axis=1)
#ypred = ypred.drop("level_0",axis=1)

randypred = randypred.reset_index()
randypred = randypred.drop("index",axis=1)

In [27]:
pred_df = pd.DataFrame(data = lreg.predict(xpred),columns=[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back'])
pred_df1 = pd.DataFrame(data = lreg1.predict(xpred),columns=[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back'])
pred_df2 = pd.DataFrame(data = lreg2.predict(randxpred),columns=[ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back'])

In [28]:
mse = np.mean((pred_df - ypred)**2)
mse1 = np.mean((pred_df1 - ypred)**2)
mse2 = np.mean((pred_df2 - randypred)**2)

In [29]:
mse,mse1,mse2

(attached_mic                    2.986673e-32
 recording_quality               4.675389e-02
 noise_handling                  4.307722e-02
 Impedance                       2.253106e-02
 sensitivity(dbv)                8.009823e-02
 accuracy                        6.303296e-02
 weighted_harmonic_distortion    6.219930e-02
 soundstage                      2.372190e-02
 imaging                         5.197113e-02
 noise_isolation                 1.546315e-01
 wireless                        5.861593e-02
 over_ear_style                  5.925938e-32
 on_ear_style                    0.000000e+00
 in_ear_style                    1.255345e-31
 open_back                       2.361192e-01
 close_back                      2.375394e-01
 semiclosed_back                 1.719079e-01
 dtype: float64, attached_mic                    0.250000
 recording_quality               0.154690
 noise_handling                  0.141808
 Impedance                       0.099650
 sensitivity(dbv)                0

In [30]:
coeff = pd.DataFrame(columns= X.columns, data=lreg.coef_)
coeff

,game_use,rec_use,studio_use,exercise_use,office_use,podcast_use,travel_use,other_use,sound_quality_importance,positional_importance,loud_importance,mic_importance,wireless_importance,in_ear_importance,on_ear_importance,over_ear_importance,no_pref_oninover
0,2.730136e-01,8.613030e-04,-7.269864e-01,-9.650494e-02,-1.410461e-01,0.000000e+00,-9.838853e-02,-6.698252e-03,2.046974e-16,-4.857226e-16,-2.081668e-16,1.179612e-16,-4.579670e-16,2.947987e-01,0.0,-3.141545e-01,0.0
1,3.031348e+13,-1.438265e+11,3.031348e+13,3.129573e+13,3.873352e+13,3.417969e-02,3.161026e+13,1.118522e+12,4.553223e-01,1.367188e-02,-1.708984e-03,-3.125000e-02,9.472656e-02,-3.404693e+13,0.0,3.727910e+13,0.0
2,-1.324480e+13,6.284177e+10,-1.324480e+13,-1.367397e+13,-1.692375e+13,-2.001953e-02,-1.381140e+13,-4.887131e+11,1.706543e-01,6.005859e-02,1.562500e-02,2.099609e-02,3.955078e-02,1.487605e+13,0.0,-1.628827e+13,0.0
3,1.486074e+12,-7.050882e+09,1.486074e+12,1.534227e+12,1.898854e+12,5.401611e-01,1.549647e+12,5.483388e+10,6.942749e-03,-1.368408e-01,2.402496e-02,-1.222229e-01,-2.081299e-02,-1.669101e+12,0.0,1.827553e+12,0.0
4,-4.230820e+13,2.007371e+11,-4.230820e+13,-4.367911e+13,-5.405996e+13,3.417969e-02,-4.411810e+13,-1.561109e+12,-4.150391e-01,1.230469e-01,1.052246e-01,2.343750e-02,1.152344e-01,4.751894e+13,0.0,-5.203004e+13,0.0
5,2.537613e+13,-1.204006e+11,2.537613e+13,2.619840e+13,3.242475e+13,-4.687500e-02,2.646170e+13,9.363411e+11,5.065918e-01,1.142578e-01,-1.179199e-01,8.789062e-02,2.246094e-02,-2.850150e+13,0.0,3.120722e+13,0.0
6,2.975078e+13,-1.411567e+11,2.975078e+13,3.071479e+13,3.801452e+13,-4.101562e-02,3.102349e+13,1.097759e+12,1.191406e-01,5.273438e-02,-1.010742e-01,6.542969e-02,5.957031e-02,-3.341493e+13,0.0,3.658710e+13,0.0
7,5.219331e+12,-2.476384e+10,5.219331e+12,5.388454e+12,6.669082e+12,-4.870605e-02,5.442610e+12,1.925855e+11,9.387207e-02,-7.397461e-02,-1.357422e-01,2.783203e-02,9.643555e-02,-5.862152e+12,0.0,6.418662e+12,0.0
8,5.831069e+12,-2.766632e+10,5.831069e+12,6.020014e+12,7.450740e+12,1.452637e-01,6.080518e+12,2.151577e+11,3.671265e-02,-8.764648e-02,-2.137146e-01,5.078125e-02,2.685547e-02,-6.549233e+12,0.0,7.170970e+12,0.0
9,-6.122808e+13,2.905051e+11,-6.122808e+13,-6.321206e+13,-7.823514e+13,1.699219e-01,-6.384737e+13,-2.259224e+12,-5.268555e-01,2.500000e-01,-8.740234e-02,2.539062e-02,-2.519531e-01,6.876902e+13,0.0,-7.529746e+13,0.0


In [31]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_shape=(17,), activation='relu'),
  #tf.keras.layers.Dense(516, activation='relu'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='linear'),
  #tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(108, activation='linear'),
  tf.keras.layers.Dense(17)
])

In [32]:
opt = tf.keras.optimizers.Adam(.001)
mae = tf.keras.losses.MeanAbsoluteError()

model.compile(optimizer=opt,metrics=['accuracy'],loss=mae)



In [33]:
r=model.fit(randx,randy, epochs=1000)

ep - loss: 0.2077 - accuracy: 0.1236
Epoch 203/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2077 - accuracy: 0.1245
Epoch 204/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2077 - accuracy: 0.1254
Epoch 205/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2076 - accuracy: 0.1236
Epoch 206/1000
250/250 [==============================] - 0s 2ms/step - loss: 0.2076 - accuracy: 0.1269
Epoch 207/1000
250/250 [==============================] - 0s 2ms/step - loss: 0.2076 - accuracy: 0.1241
Epoch 208/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2077 - accuracy: 0.1251
Epoch 209/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2076 - accuracy: 0.1249
Epoch 210/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2076 - accuracy: 0.1243
Epoch 211/1000
250/250 [==============================] - 0s 1ms/step - loss: 0.2075 - accuracy: 0.1248
Epoch 212/1000
250/250 [===

KeyboardInterrupt: 

In [34]:
# Plot the loss
plt.plot(r.history['loss'], label='loss')

NameError: name 'r' is not defined

In [35]:
plt.plot(r.history['accuracy'], label='accuracy')

NameError: name 'r' is not defined

In [36]:
aanpreds = model.predict(xpred)
randaanpreds = model.predict(randxpred)
preds = model.predict(x)

In [37]:
preddf = pd.DataFrame(data=aanpreds, columns = [ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back'])
predrand = pd.DataFrame(data=randaanpreds, columns = [ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back'])
predset = pd.DataFrame(data=preds, columns = [ 'attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back'])

In [38]:
predset

,attached_mic,recording_quality,noise_handling,Impedance,sensitivity(dbv),accuracy,weighted_harmonic_distortion,soundstage,imaging,noise_isolation,wireless,over_ear_style,on_ear_style,in_ear_style,open_back,close_back,semiclosed_back
0,0.000307,0.001822,0.419454,0.186553,0.856798,0.688130,0.433724,0.544512,0.618112,0.559364,0.002598,0.024724,0.000173,0.001185,-0.001164,0.879670,0.000917
1,0.001906,0.001961,0.499849,0.082434,0.649965,0.735481,0.512960,0.817588,0.340001,0.749867,0.002586,1.214768,0.000115,0.001396,-0.000229,0.921870,0.001254
2,0.000255,0.000924,0.467086,0.123498,0.402629,0.554614,0.701856,0.633798,0.515059,0.389046,0.000613,0.439493,0.000845,0.000360,0.000498,0.995797,-0.000074
3,0.000193,0.001409,0.443647,0.105776,0.677223,0.429498,0.669919,0.655458,0.678164,0.224511,0.001834,0.441222,0.000734,0.001286,-0.000267,1.001109,-0.000633
4,0.000193,0.001409,0.443647,0.105776,0.677223,0.429498,0.669919,0.655458,0.678164,0.224511,0.001834,0.441222,0.000734,0.001286,-0.000267,1.001109,-0.000633
5,0.000193,0.001409,0.443647,0.105776,0.677223,0.429498,0.669919,0.655458,0.678164,0.224511,0.001834,0.441222,0.000734,0.001286,-0.000267,1.001109,-0.000633
6,0.000307,0.001822,0.419454,0.186553,0.856798,0.688130,0.433724,0.544512,0.618112,0.559364,0.002598,0.024724,0.000173,0.001185,-0.001164,0.879670,0.000917
7,0.001906,0.001961,0.499849,0.082434,0.649965,0.735481,0.512960,0.817588,0.340001,0.749867,0.002586,1.214768,0.000115,0.001396,-0.000229,0.921870,0.001254
8,0.000255,0.000924,0.467086,0.123498,0.402629,0.554614,0.701856,0.633798,0.515059,0.389046,0.000613,0.439493,0.000845,0.000360,0.000498,0.995797,-0.000074
9,0.000193,0.001409,0.443647,0.105776,0.677223,0.429498,0.669919,0.655458,0.678164,0.224511,0.001834,0.441222,0.000734,0.001286,-0.000267,1.001109,-0.000633


In [39]:
mse = np.mean((aanpreds - ypred)**2)
mse1 =np.mean((randaanpreds - randypred)**2)

In [40]:
mse,mse1

(attached_mic                    4.993991e-01
 recording_quality               2.629824e-01
 noise_handling                  2.459394e-01
 Impedance                       1.329961e-01
 sensitivity(dbv)                1.354811e-01
 accuracy                        8.233864e-02
 weighted_harmonic_distortion    2.738057e-01
 soundstage                      1.408889e-01
 imaging                         1.206647e-01
 noise_isolation                 9.458200e-02
 wireless                        1.719521e-01
 over_ear_style                  5.549490e-01
 on_ear_style                    1.484874e-07
 in_ear_style                    1.152364e-01
 open_back                       6.541559e-01
 close_back                      8.374821e-01
 semiclosed_back                 1.921639e-01
 dtype: float64, attached_mic                    0.201155
 recording_quality               0.090777
 noise_handling                  0.081329
 Impedance                       0.044057
 sensitivity(dbv)                0